In [ ]:
# Data Manipulation Libraries
import pandas as pd
import numpy as np
import datetime

# Vizualization Libraries
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

# pre-processing
from sklearn.preprocessing import StandardScaler

# ML model Libraries
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import mean_squared_error

#sklearn
from sklearn.preprocessing import MinMaxScaler

#statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.statespace.sarimax import SARIMAX
!pip install pmdarima
from pmdarima.arima import auto_arima

## for Deep-learing:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM, RepeatVector, TimeDistributed, Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout

import warnings
warnings.filterwarnings('ignore')

In [ ]:
retail = pd.read_csv('/content/drive/My Drive/TheGreatIndianHiringHackathon/Train.csv')
retail_test = pd.read_csv('/content/drive/My Drive/TheGreatIndianHiringHackathon/Test.csv')
retail_data = retail.copy()
retail_test_data = retail_test.copy()

#Drop Duplicate rows
retail_data.drop_duplicates(subset=None, keep='first', inplace=True)
#Only dropped one outlier
retail_data.drop(retail_data.loc[retail_data['UnitPrice']>35000,:].index,inplace=True)

# #No missing values
# import missingno as msno
# msno.matrix(retail_test)
# print(retail_test.isna().sum())
#Seperate Categorical and Numerical Columns
cat_cols = retail_data.select_dtypes(include=['object','category']).columns.tolist()
print(cat_cols)

num_cols = retail_data.select_dtypes(include=['int64','float64']).columns.tolist()
print(num_cols)

retail_data.head()

['InvoiceDate']
['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'UnitPrice', 'CustomerID', 'Country']


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,6141,1583,144,3,2011-05-06 16:54:00,3.75,14056.0,35
1,6349,1300,3682,6,2011-05-11 07:35:00,1.95,13098.0,35
2,16783,2178,1939,4,2011-11-20 13:20:00,5.95,15044.0,35
3,16971,2115,2983,1,2011-11-22 12:07:00,0.83,15525.0,35
4,6080,1210,2886,12,2011-05-06 09:00:00,1.65,13952.0,35


In [ ]:
# sorted = retail_data.sort_values(by='InvoiceDate')
# grouped = retail_data.groupby('StockCode')['UnitPrice']

In [ ]:
# fig = px.line(sorted,x=sorted['InvoiceDate'],y='UnitPrice',color='StockCode')
# fig.show()
# peculiar_stock = ['3683','3681','3678','3680','3679','3680','3678','1511']
# sc_2384 = sorted.loc[sorted['StockCode']==2384,:]
# fig = px.line(sc_2384,x=sc_2384['InvoiceDate'],y='UnitPrice')
# fig.show()

In [ ]:
truevalues = []
for i in retail_test_data['StockCode'].values:
  if i in [3683,3681,3678,3680,3679,3680,3678,1511]:
    truevalues.append({i:retail_test_data[retail_test_data['StockCode'] == i]})

In [ ]:
#Common Methods

#Quantity cant be negative
def make_quantity_abs(df):
  df['Quantity'] = df['Quantity'].abs()

def drop_columns(df1,df2,columnList):
  df1.drop(columns=columnList,inplace=True)
  df2.drop(columns=columnList,inplace=True)

def set_invoicedate_index_sort(df):
  df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
  df.set_index('InvoiceDate',inplace=True)
  df.index = pd.to_datetime(df.index.strftime('%Y-%m-%d %H:%M:%S'))
  df.sort_index(inplace=True)
  df.resample()
  return df

def change_to_appropriate_type(df):
  df['StockCode'] = df['StockCode'].astype('string')
  df['CustomerID'] = df['CustomerID'].astype('int')
  df['CustomerID'] = df['CustomerID'].astype('string')

def code_country(df):
  df.loc[df['Country']=='35','Country'] = 'country35'
  df.loc[df['Country']!='country35','Country'] = 'OtherCountry'

def replace_stockcode_with_quantity(train_df,test_df):
  #Highest Quantity purchase
  train_temp = train_df.copy()
  train_quantity_sum = train_temp[['StockCode','Quantity']].groupby(['StockCode']).sum().copy()
  train_quantity_sum.index = train_quantity_sum.index.astype('int')
  train_quantity_sum.sort_values(by='StockCode',inplace=True)

  test_temp = test_df.copy()
  test_quantity_sum = test_temp[['StockCode','Quantity']].groupby(['StockCode']).sum().copy()
  test_quantity_sum.index = test_quantity_sum.index.astype('int')
  test_quantity_sum.sort_values(by='StockCode',inplace=True)

  #replacing train with train quantity sum
  train_df = pd.DataFrame(train_df)
  train_df.replace(train_quantity_sum.to_dict(),inplace=True)
  train_df.rename(columns={'StockCode':'StockCodeQuantity'},inplace=True)

  #replacing test with train quantity sum
  train_df = pd.DataFrame(test_df)
  test_df.replace(train_quantity_sum.to_dict(),inplace=True)
  #balance if any
  test_df.replace(test_quantity_sum.to_dict(),inplace=True)
  test_df.rename(columns={'StockCode':'StockCodeQuantity'},inplace=True)

  train_df.drop(columns=['Quantity'],inplace=True)
  test_df.drop(columns=['Quantity'],inplace=True)

def add_unique_customer_count_per_day(df):
  #Adding unique Customer count
  df['UniqueCustomerCount'] = [datetime.datetime.strftime(df.index[dt], '%Y-%m-%d') for dt in range(len(df.index))]
  replace_uniqueCustomerCount = df.groupby('UniqueCustomerCount')['CustomerID'].count().to_dict()
  df['UniqueCustomerCount'].replace(replace_uniqueCustomerCount,inplace=True)

def convert_column_to_dummies(df,colname):
  dummies = pd.get_dummies(df[colname])

  for col in dummies.columns:
    df[col] = dummies[col]

  df.drop(columns=[colname],inplace=True)

def categorize_y(y):
  y = np.round(y)
  y = y.astype(int)
  for i in range(len(y)):
    if int(y[i]) == 1:
      y[i] = '1'
    elif int(y[i]) == 2:
      y[i] = '2'
    elif int(y[i]) == 3:
      y[i] = '3'
    elif int(y[i]) == 4:
      y[i] = '4'
    elif int(y[i]) == 5:
      y[i] = '5'
    elif int(y[i]) == 6:
      y[i] = '6'
    elif int(y[i]) == 7:
      y[i] = '7'
    elif int(y[i]) == 8:
      y[i] = '8'
    elif int(y[i]) == 9:
      y[i] = '9'
    elif int(y[i]) == 10:
      y[i] = '10'
    elif int(y[i]) > 11 & int(y[i]) <= 15:
      y[i] = '15'
    elif int(y[i]) > 15 & int(y[i]) <= 20:
      y[i] = '20'
    elif int(y[i]) > 20 & int(y[i]) <= 50:
      y[i] = '50'
    elif int(y[i]) > 50 & int(y[i]) <= 100:
      y[i] = '100'
    else:
      y[i] = '500'
  
  return y

scaler = StandardScaler(with_mean=True,with_std=True)
def perform_scaling(toscale):
  global scaler
  scaler = StandardScaler(with_mean=True,with_std=True)
  return pd.DataFrame(scaler.fit_transform(toscale),columns=toscale.columns,index=toscale.index)

def execute_known_y_test(df,test_size=0.3,shuffleBool=False):
  X = df[['StockCodeQuantity','UniqueCustomerCount',	'OtherCountry',	'country35']]
  y = df['UnitPrice']
  X[['StockCodeQuantity','UniqueCustomerCount']] = perform_scaling(df[['StockCodeQuantity','UniqueCustomerCount']])
  y = categorize_y(y)
  return train_test_split(X, y, test_size=test_size, random_state=42,shuffle=shuffleBool)

    
def execute_unknown_y_test(df1,df2):
  ssc = StandardScaler(with_mean=True,with_std=True)

  x_train = df1[['UniqueCustomerCount']]
  x_train[['UniqueCustomerCount']] = perform_scaling(x_train[['UniqueCustomerCount']])
  y_train = perform_scaling(df1)
  # y_train = categorize_y(y_train)

  #retail_test_data
  x_test = df2[['UniqueCustomerCount']]
  x_test[['UniqueCustomerCount']] = perform_scaling(x_test[['UniqueCustomerCount']])

  return x_train, y_train, x_test

def save_export_predicted_values(test_data,y_pred,save_name):

  test_data['UnitPrice'] = y_pred
  sorted_predicted_dataframe = test_data.sort_values('order')
  UnitPrice = pd.DataFrame(sorted_predicted_dataframe['UnitPrice'].astype('int').values,columns=['UnitPrice'])
  UnitPrice.to_csv('/content/drive/My Drive/TheGreatIndianHiringHackathon/'+save_name+'.csv', index=False, index_label=None)

def match_predictions(predictions,test_data,save_name):
  predictions = pd.DataFrame(predictions,columns=['Predictions'])
  predictions['Predictions'] = np.round(predictions['Predictions'],decimals=2)
  predictions['pred'] = predictions.index.strftime('%Y-%m-%d').astype('string')
  dictionary_map = pd.Series(predictions['Predictions'].values,index=predictions['pred']).to_dict()

  test_data['UnitPrice'] = test_data.index.strftime('%Y-%m-%d').astype('string')
  test_data['UnitPrice'].replace(dictionary_map,inplace=True)
  sorted_predicted_dataframe = test_data.sort_values('order')
  UnitPrice = pd.DataFrame(sorted_predicted_dataframe['UnitPrice'].astype('float').values,columns=['UnitPrice'])
  UnitPrice.to_csv('/content/drive/My Drive/TheGreatIndianHiringHackathon/'+save_name+'.csv', index=False, index_label=None)

def remove_non_business_hours(df1):
  hour_resampled_data_index = df1.index
  to_drop_index = []
  for t in hour_resampled_data_index:
    if t.hour < 8 | t.hour > 18:
      to_drop_index.append(t)

  df1.drop(index=to_drop_index,inplace=True)


In [ ]:
retail_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,6141,1583,144,3,2011-05-06 16:54:00,3.75,14056.0,35
1,6349,1300,3682,6,2011-05-11 07:35:00,1.95,13098.0,35
2,16783,2178,1939,4,2011-11-20 13:20:00,5.95,15044.0,35
3,16971,2115,2983,1,2011-11-22 12:07:00,0.83,15525.0,35
4,6080,1210,2886,12,2011-05-06 09:00:00,1.65,13952.0,35


In [ ]:
make_quantity_abs(retail_data)
make_quantity_abs(retail_test_data)

drop_columns(retail_data,retail_test_data,['Description','InvoiceNo','Country'])

retail_data = set_invoicedate_index_sort(retail_data)
retail_test_data['order'] = range(0,retail_test_data.shape[0])
retail_test_data = set_invoicedate_index_sort(retail_test_data)

change_to_appropriate_type(retail_data)
change_to_appropriate_type(retail_test_data)

# code_country(retail_data)
# code_country(retail_test_data)

#replace_stockcode_with_quantity(retail_data,retail_test_data)

#add_unique_customer_count_per_day(retail_data)
#add_unique_customer_count_per_day(retail_test_data)

#convert_column_to_dummies(retail_data,'Country')
#convert_column_to_dummies(retail_test_data,'Country')

hour_resampled_retail_data = retail_data.resample(rule='H', closed="right", label="left", base=23).mean().fillna(0)
hour_resampled_retail_test_data = retail_test_data.resample(rule='H', closed="right", label="left", base=23).mean().fillna(0)

remove_non_business_hours(retail_data)
#remove_non_business_hours(retail_test_data)

In [ ]:
stock_code_dummies = pd.get_dummies(retail_data['StockCode'])
stock_code_dummies.head()

,0,1,10,100,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,101,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,102,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,103,1030,1031,1032,...,963,964,965,966,967,968,969,97,970,971,972,973,974,975,976,977,978,979,98,980,981,982,983,984,985,986,987,988,989,99,990,991,992,993,994,995,996,997,998,999
InvoiceDate,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-12-01 08:26:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2010-12-01 08:26:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2010-12-01 08:26:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2010-12-01 08:26:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2010-12-01 08:26:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
 retail_data = pd.concat([retail_data, pd.get_dummies(retail_data['StockCode'])], axis=1)

In [ ]:
# Create an offset 
fig = px.line(hour_resampled_retail_data, x=hour_resampled_retail_data.index,y='UnitPrice')
fig.show()

In [ ]:
retail_data_resampled = hour_resampled_retail_data['UnitPrice']
plot_pacf(retail_data_resampled,alpha=0.05,lags=4)
plt.show()

In [ ]:
plot_pacf(retail_data_resampled,alpha=0.05,lags=4)
plt.show()

In [ ]:
# bic_aic_sum = []
# result = []
# for i in range(1,3):
#     for k in range(1,3):
#         newarray = retail_data_resampled-retail_data_resampled.shift(k)
#         ma_mod = ARMA(newarray.dropna(), order=(i,k))
#         ma_res = ma_mod.fit()
#         result.append({'aic':ma_res.aic,'bic':ma_res.bic,'bic_aic_sum':ma_res.aic+ma_res.bic,'res':ma_res,'ar_index':i,'ma_index':k})

In [ ]:
# #bic aic sum lowest for arima(1,0,1)
# print(result[0]['res'].summary())

In [ ]:
# model = sm.tsa.ARMA(retail_data_resampled, (1, 1))
# results = model.fit()
# predictions = results.predict(start=retail_data_resampled.index[0],end=retail_data_resampled.index[-1])
# plt.figure(figsize=(20,5))
# ax = plt.plot(retail_data_resampled)
# ax = plt.plot(predictions)
# plt.show()

In [ ]:
# x_train, x_test, y_train, y_test = execute_known_y_test(retail_data,0.3,False)

# step_wise = auto_arima(y_train, exogenous= x_train,
#                        start_p=1, start_q=1, 
#                        max_p=7, max_q=7, d=1, max_d=7, 
#                        trace=True, error_action='ignore', suppress_warnings=True, stepwise=True) #seasonal=True/ trend='c'



Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=120.95 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=2670770.627, Time=7.48 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=2613848.562, Time=15.23 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=101.67 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=2670768.800, Time=73.47 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=2590577.225, Time=11.09 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=2577799.714, Time=22.06 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=2569754.189, Time=27.74 sec
 ARIMA(5,1,0)(0,0,0)[0] intercept   : AIC=2564327.959, Time=34.78 sec
 ARIMA(6,1,0)(0,0,0)[0] intercept   : AIC=2560287.625, Time=44.12 sec
 ARIMA(7,1,0)(0,0,0)[0] intercept   : AIC=2557165.078, Time=47.20 sec
 ARIMA(7,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=697.01 sec
 ARIMA(6,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=639.02 sec





In [ ]:
# plt.rcParams['agg.path.chunksize'] = 10000
# plt.rcParams.update({'figure.figsize': (20,10)})
# frequency = 60
# # multiplicative = sm.tsa.seasonal_decompose(retail_data['UnitPrice'].to_frame(),period=frequency,model='multiplicative')
# # multiplicative.plot().suptitle('Multiplicative Decompose', fontsize=22)
# # plt.show()

# additive = sm.tsa.seasonal_decompose(retail_data['UnitPrice'].to_frame(),freq=frequency,model='additive')
# additive.plot().suptitle('Additive Decompose', fontsize=22)
# plt.show()
# plt.rcParams.update({'figure.figsize': (15,5)})

In [ ]:
# x_train, y_train, x_test = execute_unknown_y_test(hour_resampled_retail_data,hour_resampled_retail_test_data)
# forecast = sarimax_results.forecast(steps=test_size-1, exog=test_X[3])
# x_train

In [ ]:
hour_resampled_retail_data

In [ ]:
# converting into supervised learning
# train_time_samples = x_train.index
# test_time_samples = x_test.index
# extra = []
# for i in test_time_samples:
#   if(i not in train_time_samples):
#     extra.append(i)

# print(extra)
#all test samples are there in train samples

In [ ]:
print(scaled_y)

In [ ]:
# ssc = StandardScaler(with_mean=True,with_std=True)
# minmaxscaler = MinMaxScaler()
# scaled_y = minmaxscaler.fit_transform(hour_resampled_retail_data['UnitPrice'].values.reshape(1,-1))
# scaled_y

In [ ]:
def series_to_supervised(data, n_in=7, n_out=1, dropnan=True):
	n_vars = 1
	dff = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(dff.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	# for i in range(0, n_out):
	# 	cols.append(dff.shift(-i))
	# 	if i == 0:
	# 		names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
	# 	else:
	# 		names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
reframed = series_to_supervised(hour_resampled_retail_data['UnitPrice'].values, n_in=7, n_out=1, dropnan=True)
reframed['actual'] = hour_resampled_retail_data['UnitPrice'].values[:-7]
reframed

In [ ]:
# split into train and test sets
values = reframed.values

n_train_time = len(reframed)
train = values[:n_train_time, :]
test = values[n_train_time:, :]
##test = values[n_train_time:n_test_time, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

test_X = train_X
test_y = train_y

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape) 

In [ ]:
# model = Sequential()
# model.add(LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2])))
# model.add(Dropout(0.2))
# #    model.add(LSTM(70))
# #    model.add(Dropout(0.3))
# model.add(Dense(1))
# model.compile(loss='mean_squared_error', optimizer='adam')

#CNN + LSTM
model_cnn_lstm = Sequential()
model_cnn_lstm.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None,train_X.shape[1], train_X.shape[2])))
model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=1)))
model_cnn_lstm.add(TimeDistributed(Flatten()))
model_cnn_lstm.add(LSTM(50, activation='relu'))
model_cnn_lstm.add(Dense(1))
model_cnn_lstm.compile(loss='mean_squared_error', optimizer='adam')

# fit network
history = model.fit(train_X, train_y, epochs=20, batch_size=70, validation_data=(test_X, test_y), verbose=2, shuffle=False)

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], 7))
# invert scaling for forecast
inv_yhat = np.concatenate((yhat, test_X[:, -7:]), axis=1)
# inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_y, test_X[:, -7:]), axis=1)
# inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
result = pd.DataFrame(inv_yhat,index=x_train.index[:-7])

In [ ]:
result

In [ ]:
ax = plt.plot(inv_y)
ax = plt.plot(inv_yhat)
plt.show()

In [ ]:
match_test_data = retail_test_data.index
hour_converted = []
for i in match_test_data:
  hour_converted.append(i.strftime('%Y-%m-%d %H:00:00'))

new_result = result
new_result['map'] = new_result.index.strftime('%Y-%m-%d %H:00:00')
new_result.columns = ['c','map']
new_result.index = new_result['map']
replace_dict = new_result['c'].to_dict()
retail_test_data['predictions'] = retail_test_data.index.strftime('%Y-%m-%d %H:00:00')
retail_test_data['predictions'].replace(replace_dict,inplace=True)
future_predictions = {'2011-12-09 08:00:00':2.23021,'2011-12-09 09:00:00':2.23021,'2011-12-09 10:00:00':2.23021,'2011-12-09 11:00:00':2.23021,'2011-12-09 12:00:00':2.23021}
retail_test_data['predictions'].replace(future_predictions,inplace=True)


#

In [ ]:
 retail_test_data['predictions'] = np.round(retail_test_data['predictions'].astype(float),decimals=2)
retail_test_data.sort_values('order',inplace=True)
UnitPrice = pd.DataFrame(retail_test_data['predictions'].astype('float').values,columns=['UnitPrice'])
UnitPrice.to_csv('/content/drive/My Drive/TheGreatIndianHiringHackathon/'+'kkcnnlstm05'+'.csv', index=False, index_label=None)

In [ ]:
fg = pd.read_csv('/content/kklstm01.csv')
fg.shape

In [ ]:
c = retail_data.sort_index()
c.tail()

In [ ]:
h = retail_test_data.sort_index()
h.tail(40)

In [ ]:
# #SARIMAX
# model = SARIMAX(y_train, order=(3,1,0),seasonal_order=(1,0,0,7), seasonal=True, enforce_invertibility=False, enforce_stationarity=False,exog=x_train)
# sarimax_results = model.fit()
# predictions = sarimax_results.predict(start=x_train.index[0], end=x_train.index[-1],exog=x_test)
# plt.figure(figsize=(20,5))
# ax = plt.plot(y_train)
# ax = plt.plot(predictions)
# plt.show()
# #One thing that could be done is by distributing the day prediction accross the time stamps of that day. could reduce the error.
# mean_squared_error(y_train,predictions)
# # mean_prediction=predictions
# # mean_prediction.loc[:] = np.round(y_train.mean(),decimals=2)
# # match_predictions(mean_prediction,retail_test_data,'kkmean10')